<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_9_1__OdpytywanieBazyDanych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qqU langchain_community crewai crewai_tools  pydantic-core==2.23.4 pydantic-settings==2.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.8/170.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.7/169.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 29.4 MB/s eta 0:00:00


In [ ]:
# !pip install -qU  langchain-core==0.2.9 langchain-community==0.2.5 'crewai[tools]'==0.32.0

Ten kod instaluje trzy pakiety Pythona przy użyciu narzędzia pip:

- langchain_community - bibliotekę do przetwarzania języka naturalnego i budowania aplikacji wykorzystujących duże modele językowe
- crewai - framework do tworzenia zespołów agentów AI, które mogą współpracować ze sobą
- crewai_tools - zestaw narzędzi rozszerzających możliwości crewai

Flaga -qq wymusza bardzo cichy tryb instalacji (suppressing output), U wymusza aktualizację pakietów do najnowszych wersji.

In [ ]:
# Standard library imports
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

# Third-party imports
import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

Ten kod zawiera importy potrzebnych bibliotek i modułów, podzielone na dwie sekcje:

Z biblioteki standardowej Pythona:
- json - do obsługi formatu JSON
- os - do operacji na systemie operacyjnym
- sqlite3 - do pracy z bazami SQLite
- dataclasses - narzędzia do tworzenia klas danych
- datetime - do obsługi dat i czasu
- pathlib - do operacji na ścieżkach plików
- textwrap - do formatowania tekstu
- typing - do dodawania typów zmiennych

Z zewnętrznych bibliotek:
- pandas - do analizy danych
- moduły z crewai - do tworzenia i zarządzania agentami AI
- narzędzia z crewai_tools
- google.colab - do integracji z Google Colab
- komponenty z langchain - do pracy z modelami językowymi:
  - narzędzia do obsługi baz SQL
  - system callbacków
  - szablony promptów
  - integracja z OpenAI

Kod wykorzystuje typowanie statyczne przez importy z modułu typing (Any, Dict, List, Tuple, Union), co zwiększa czytelność i ułatwia debugowanie.

In [ ]:
class CFG:
    model = 'gpt-4o-mini'
    temp = 0.1

In [ ]:

os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')


In [ ]:
llm=ChatOpenAI(model_name= CFG.model, temperature= CFG.temp)

Ten kod tworzy instancję modelu językowego OpenAI:

- llm - zmienna przechowująca instancję modelu
- ChatOpenAI() - konstruktor tworzący połączenie z API OpenAI
- model_name pobiera nazwę modelu z konfiguracji (CFG.model)
- temperature określa losowość/kreatywność odpowiedzi (z CFG.temp)

Temperature to parametr między 0 a 2, gdzie:
- niskie wartości (bliskie 0) dają bardziej przewidywalne, spójne odpowiedzi
- wysokie wartości zwiększają kreatywność i różnorodność odpowiedzi

# Data

Źródło: https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023

In [ ]:
# plik -> baza
df = pd.read_csv("/content/ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [ ]:
connection = sqlite3.connect("salaries.db")
df.to_sql(name="salaries", con=connection)

3755

Ten kod wykonuje dwie operacje związane z bazą danych:

1. `sqlite3.connect("salaries.db")` tworzy połączenie z bazą SQLite o nazwie "salaries.db". Jeśli baza nie istnieje, zostanie automatycznie utworzona w bieżącym katalogu.

2. `df.to_sql()` zapisuje dane z DataFrame'u do tabeli w bazie SQLite:
   - name="salaries" określa nazwę tworzonej tabeli
   - con=connection wskazuje połączenie z bazą, do której dane mają być zapisane

Operacja automatycznie przekształca strukturę DataFrame'u na odpowiadającą jej tabelę SQL, zachowując typy danych i nazwy kolumn.

In [ ]:
# połączenie do bazy
db = SQLDatabase.from_uri("sqlite:///salaries.db")


Ten kod tworzy obiekt bazy danych przy użyciu SQLDatabase z biblioteki langchain:

`SQLDatabase.from_uri()` nawiązuje połączenie do bazy SQLite używając URI:
- "sqlite:///" to prefix wskazujący na lokalną bazę SQLite
- "salaries.db" to nazwa pliku bazy danych

Obiekt `db` pozwala na wykonywanie operacji na bazie danych w sposób kompatybilny z narzędziami langchain, które mogą być wykorzystane przez agentów AI do analizy i manipulacji danymi.

# Agentura


## Narzędzia

In [ ]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

Using Tool: list_tables


'salaries'

Ten kod definiuje i wykonuje narzędzie do listowania tabel w bazie danych:

1. `@tool("list_tables")` to dekorator, który rejestruje funkcję jako narzędzie możliwe do użycia przez agentów AI.

2. Funkcja `list_tables()`:
   - zwraca typ string (-> str)
   - wykorzystuje `ListSQLDatabaseTool` z langchain
   - parametr db=db przekazuje utworzone wcześniej połączenie z bazą
   - invoke("") wywołuje narzędzie z pustym parametrem

3. `list_tables.run()` wykonuje zdefiniowane narzędzie, które wyświetli nazwy wszystkich tabel dostępnych w bazie danych.

In [ ]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print(tables_schema.run("salaries"))

Using Tool: tables_schema

CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2023	SE	FT	Principal Data Scientist	80000	EUR	85847	ES	100	ES	L
1	2023	MI	CT	ML Engineer	30000	USD	30000	US	100	US	S
2	2023	MI	CT	ML Engineer	25500	USD	25500	US	100	US	S
*/


Ten kod definiuje i używa narzędzie do analizy struktury tabel:

1. Dekorator `@tool("tables_schema")` rejestruje funkcję jako narzędzie o nazwie "tables_schema"

2. Funkcja `tables_schema()`:
   - przyjmuje parametr `tables` jako string z nazwami tabel oddzielonymi przecinkami
   - zwraca string z informacjami o strukturze tabel
   - używa `InfoSQLDatabaseTool` z langchain do pobrania informacji o tabelach
   - parametr db=db przekazuje połączenie do bazy

3. Dokumentacja funkcji w potrójnych cudzysłowach wyjaśnia:
   - format wejścia (nazwy tabel oddzielone przecinkami)
   - typ wyniku (schemat i przykładowe wiersze)
   - zalecenie sprawdzenia istnienia tabel przez `list_tables`
   - przykład wejścia: "table1, table2, table3"

4. `print(tables_schema.run("salaries"))` wyświetla strukturę tabeli "salaries", pokazując:
   - nazwy kolumn
   - typy danych
   - przykładowe wiersze z tabeli

In [ ]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

Using Tool: execute_sql


"[(0, 2023, 'SE', 'FT', 'Principal Data Scientist', 80000, 'EUR', 85847, 'ES', 100, 'ES', 'L'), (1, 2023, 'MI', 'CT', 'ML Engineer', 30000, 'USD', 30000, 'US', 100, 'US', 'S'), (2, 2023, 'MI', 'CT', 'ML Engineer', 25500, 'USD', 25500, 'US', 100, 'US', 'S'), (3, 2023, 'SE', 'FT', 'Data Scientist', 175000, 'USD', 175000, 'CA', 100, 'CA', 'M'), (4, 2023, 'SE', 'FT', 'Data Scientist', 120000, 'USD', 120000, 'CA', 100, 'CA', 'M')]"

Ten kod definiuje i wykonuje narzędzie do wykonywania zapytań SQL:

1. Funkcja `execute_sql()`:
   - przyjmuje parametr `sql_query` zawierający zapytanie SQL
   - zwraca wynik zapytania jako string
   - używa `QuerySQLDataBaseTool` do wykonania zapytania na bazie
   - parametr db=db przekazuje połączenie do bazy

2. Wywołanie `execute_sql.run()` wykonuje przykładowe zapytanie SQL:
   - `SELECT * FROM salaries` wybiera wszystkie kolumny z tabeli salaries
   - `WHERE salary > 10000` filtruje wiersze gdzie pensja przekracza 10000
   - `LIMIT 5` ogranicza wynik do pierwszych 5 wierszy

Narzędzie to pozwala agentom AI na elastyczne odpytywanie bazy danych przy użyciu standardowego języka SQL.

In [ ]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})


check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'```sql\nSELECT * FROM salaries WHERE salary > 10000 LIMIT 5;\n```'

Ten kod definiuje narzędzie do sprawdzania poprawności zapytań SQL:

1. Funkcja `check_sql()`:
   - przyjmuje parametr `sql_query` z zapytaniem do sprawdzenia
   - wykorzystuje `QuerySQLCheckerTool`, który wymaga:
     - połączenia do bazy (db=db)
     - modelu językowego (llm=llm) do analizy zapytania

2. Wywołanie `check_sql.run()` sprawdza przykładowe zapytanie, które zawiera błędy:
   - brak FROM przed nazwą tabeli
   - niepoprawna składnia `table = salaries`
   - ogólnie nieprawidłowa struktura zapytania

Narzędzie działa jak automatyczny walidator, który sprawdza składnię i logikę zapytań SQL przed ich faktycznym wykonaniem, co pomaga uniknąć błędów w czasie wykonania.

## Agenci

In [ ]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

Ten kod tworzy agenta AI specjalizującego się w operacjach bazodanowych:

1. `Agent()` konstruktor przyjmuje parametry:

   - role="Senior Database Developer" - określa rolę agenta
   - goal="Construct and execute SQL queries based on a request" - definiuje cel jego działania
   - backstory - opisuje profil agenta:
     - doświadczony inżynier baz danych
     - specjalista od złożonych i zoptymalizowanych zapytań
     - ma dostęp do czterech narzędzi
   
2. Agent otrzymuje narzędzia:
   - list_tables - do sprawdzania dostępnych tabel
   - tables_schema - do analizy struktury tabel
   - execute_sql - do wykonywania zapytań
   - check_sql - do walidacji zapytań

3. Dodatkowe parametry:
   - llm=llm - przypisuje model językowy
   - allow_delegation=False - wyłącza możliwość delegowania zadań

Funkcja `dedent()` usuwa wspólne wcięcia z tekstu wielowierszowego, poprawiając jego czytelność.

In [ ]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

Ten kod tworzy agenta o nazwie "data_analyst", który jest skonfigurowany jako starszy analityk danych.

Agent otrzymuje kilka kluczowych parametrów:

`role` ustawia jego rolę jako "Senior Data Analyst"

`goal` określa główne zadanie - otrzymywanie i analizowanie danych od programisty bazy danych

`backstory` definiuje jego profil i doświadczenie:
- Ma duże doświadczenie w analizie danych z użyciem Pythona
- Jego praca opiera się zawsze na dostarczonych danych
- Przedstawia wyniki w sposób jasny i konkretny
- Ma wysoką dbałość o szczegóły
- Tworzy bardzo szczegółowe analizy gdy jest to potrzebne

`llm` przypisuje model językowy do agenta

`allow_delegation=False` wyłącza możliwość delegowania zadań innym agentom


In [ ]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

Ten kod tworzy agenta o nazwie "report_writer", który działa jako redaktor raportów na poziomie senior.

`role` określa jego rolę jako "Senior Report Editor"

`goal` definiuje jego zadanie - pisanie podsumowań wykonawczych (executive summary) na podstawie pracy analityka

`backstory` określa jego kluczowe kompetencje:
- Jest znany z jasnej i skutecznej komunikacji
- Specjalizuje się w streszczaniu długich tekstów
- Tworzy punktowe podsumowania zawierające najważniejsze informacje

`llm` przypisuje model językowy do agenta

`allow_delegation=False` wyłącza możliwość delegowania zadań innym agentom

Jest to agent wyspecjalizowany w tworzeniu zwięzłych, klarownych raportów, który przekształca szczegółowe analizy w formę przystępną dla kadry zarządzającej.

## Zadania

In [ ]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

Ten kod definiuje zadanie o nazwie "extract_data", które służy do wydobywania danych z bazy.

`description` określa opis zadania - wydobycie danych potrzebnych dla zapytania {query}. Zmienna {query} będzie zastępowana konkretnym zapytaniem w momencie wykonania.

`expected_output` definiuje oczekiwany wynik jako "Database result for the query", czyli wynik zapytania z bazy danych

`agent=sql_dev` przypisuje to zadanie do agenta sql_dev, który będzie odpowiedzialny za jego wykonanie

Jest to definicja zadania wydobycia danych, które zostanie wykonane przez dewelopera SQL gdy pojawi się konkretne zapytanie do bazy danych.

In [ ]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

Ten kod definiuje zadanie o nazwie "analyze_data", które służy do analizy danych wydobytych z bazy.

`description` określa opis zadania - przeanalizowanie danych z bazy i stworzenie analizy dla zapytania {query}

`expected_output` definiuje oczekiwany wynik jako "Detailed analysis text", czyli szczegółowy tekst analizy

`agent=data_analyst` przypisuje to zadanie do wcześniej zdefiniowanego agenta data_analyst

`context=[extract_data]` dodaje zadanie extract_data jako kontekst - oznacza to, że analyze_data będzie miało dostęp do wyniku zadania extract_data (czyli do danych z bazy)

Jest to definicja zadania analitycznego, które wykorzystuje dane wydobyte przez poprzednie zadanie i tworzy na ich podstawie szczegółową analizę.

In [ ]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

Ten kod definiuje zadanie o nazwie "write_report", które służy do tworzenia podsumowania wykonawczego.

`description` określa opis zadania - napisanie krótkiego podsumowania wykonawczego na podstawie wcześniejszej analizy. Tekst musi mieć mniej niż 100 słów.

`expected_output` definiuje oczekiwany wynik jako "Markdown report", czyli raport w formacie Markdown

`agent=report_writer` przypisuje to zadanie do wcześniej zdefiniowanego agenta report_writer

`context=[analyze_data]` dodaje zadanie analyze_data jako kontekst - oznacza to, że write_report będzie miało dostęp do wyniku analizy stworzonej w poprzednim zadaniu

Jest to definicja zadania pisania raportu, które przekształca szczegółową analizę w zwięzłe podsumowanie wykonawcze w formacie Markdown.

## załoga

In [ ]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose= True,
    memory=False,
    output_log_file="crew.log",
)


# Test

In [ ]:
inputs = {
    "query": "How doescompany location, size and employee experience affect the salary?"
}

result = crew.kickoff(inputs=inputs)

# Agent: Senior Database Developer
## Task: Extract data that is required for the query How doescompany location, size and employee experience affect the salary?.


# Agent: Senior Database Developer
## Thought: I need to first identify the available tables in the database to understand where I might find the relevant data regarding company location, size, employee experience, and salary.
## Using tool: list_tables
## Tool Input: 
"{}"
## Tool Output: 
salaries


# Agent: Senior Database Developer
## Thought: Thought: The only available table is "salaries". I will now check the schema of the "salaries" table to understand its structure and see if it contains the necessary fields related to company location, size, employee experience, and salary.
## Using tool: tables_schema
## Tool Input: 
"{\"tables\": \"salaries\"}"
## Tool Output: 

CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	sa